# AP Project

In [178]:
class Arc:
    
    def __init__(self,id,t,b,c):
        self.t = t
        self.b = b
        self.c = c
        self.id = id
        
    def getT(self):
        return self.t
    
    def getB(self):
        return self.b
    
    def getC(self):
        return self.c

    def getId(self):
        return self.id
        
    def isSame(self,arc):
        if self.t == arc.getT() and self.b == arc.getB() and self.c == arc.getC():
            return True
        else:
            return False

class Node:
    
    def __init__(self,id,capacity,limit,due):
        self.id = id
        self.capacity = capacity
        self.limit = limit
        self.due = due
        self.arcs=[]
        
    def getCapacity(self):
        return self.capacity
        
    def getLimit(self):
        return self.limit
        
    def getDue(self):
        return self.due
    
    def getId(self):
        return self.id
    
    def addArc(self,id):
        if id not in self.arcs:
            self.arcs.append(id)
            
    def getArcs(self):
        return self.arcs

In [179]:
def getLimit(arcId,i,nodes):
    for node in nodes:
        if node.getId() == i:
            node.addArc(arcId)
            return node.getDue()

def getArcById(id,arrs):
    for arr in arrs:
        if arr.getId() == id:
            return arr
        
def mult(arcs1,arcs2):
    
    mat = np.zeros(arcs1.shape)
    
    for i in range(arcs1.shape[0]):
        for j in range(arcs1.shape[1]):
            if arcs1[i][j] != 0 and arcs2[i][j] != 0:
                if arcs1[i][j].isSame(arcs2[i][j]):
                    mat[i][j] = 1
                else:
                    mat[i][j] = 0
            else:
                mat[i][j] = 0
                
    return mat

In [180]:
def handlePointLine(point_line):
    '''
        handle the line of point, which has three parameters
        (number of persons, max_rate, last_time)
        return the (num_person, max_rate, last_time)(type=int)
    '''
    points_info = point_line.strip('\n').split(' ')
    if len(points_info) != 3:
        raise ValueError("The length of points must be 3!!")
    else:
        num_person = int(points_info[0])
        max_rate = int(points_info[1])
        last_time = int(points_info[2])
        return (num_person, max_rate, last_time)


def handleEdgeLine(edge_line):
    '''
        same as the last function, but the infos changes
    '''
    edge_info = edge_line.strip('\n').split(' ')
    capacity = int(edge_info[0])
    points_used = int(edge_info[1])
    if len(edge_info) != 2+2*points_used:
        raise ValueError("Lack of informations in the edges!!")
    else:
        points_dict = {}
        for i in range(2, len(edge_info), 2):
            index = int(edge_info[i])
            time_min = int(edge_info[i+1])
            points_dict[index] = time_min
        
        return (capacity, points_used, points_dict)

def getInfoUsedInEdge(point_dict):
    points_used = []
    time_min_point = []
    for key, value in point_dict.items():
        points_used.append(key)
        time_min_point.append(value)
    return points_used, time_min_point

def getPointToEdgeInfo(edges, p_num):
    '''
        get the [(edge_index: min_time), ...] for each point
    '''
    point_to_edges = []
    for cnt in range(p_num):
        point_to_edges.append({})

    for cnt in range(len(edges)):
        (_, _, p_dict) = edges[cnt]
        for key, value in p_dict.items():
            point_to_edges[key][cnt] = value

    # resort the dict by the value
    for i in range(p_num):
        point_to_edges[i] = sorted(point_to_edges[i].items(), key=lambda item:item[1])

    return point_to_edges

# open the file to read data
fo = open("test_10_25_2_11.evac", "r+")

first_line = fo.readline().strip('\n').split(' ')
point_num = int(first_line[0])
edge_num = int(first_line[1])

points = []
edges = []

for i in range(point_num):
    point = handlePointLine(fo.readline())
    points.append(point)

for i in range(edge_num):
    edge = handleEdgeLine(fo.readline())    
    edges.append(edge)
fo.close()

point_to_edges = getPointToEdgeInfo(edges, point_num)

time_edge = [-1]*edge_num

for list_item in point_to_edges:
    len_point_edge = len(list_item)
    if len_point_edge > 1:
        for i in range(len_point_edge -1):
            (edge_index, min_time) = list_item[i]
            (next_edge_index, next_min_time) = list_item[i+1]
            if time_edge[edge_index] == -1:
                time_edge[edge_index] = next_min_time - min_time
            elif time_edge[edge_index] != -1 and time_edge[edge_index] == next_min_time - min_time:
                continue
            else:
                raise ValueError("There are something wrong in the time")

print(time_edge)

[29, 4, -1, 31, 1, 10, 5, 2, 2]


In [181]:
file = open("test_10_25_2_11.evac", "r") 
nodes,arcs = file.readline().split(' ')
nodeArray=[]
for i in range(int(nodes)):
    cap,ma,due = file.readline().strip().split(' ')
    nodeArray.append(Node(i,int(cap),int(ma),int(due)))

arcArray=[]
for i in range(int(arcs)):
    arr = file.readline().strip().split(' ')
    cap = int(arr[0])
    nb = int(arr[1])
    
    minlimit = 100000000000
    for j in range(nb):
        newLimit = getLimit(i,int(arr[2+(2*j)]),nodeArray)
        if minlimit >= newLimit:
            minlimit = newLimit
    arcArray.append(Arc(i,2,minlimit,cap))

In [182]:
import numpy as np
arcs = []

for node in nodeArray:
    e = []
    for arc in node.getArcs():
        lineArc = [getArcById(arc,arcArray)]
        for i in range(len(nodeArray)-1):
            lineArc.append(0)
        lineArc = np.array(lineArc)
        e.append(lineArc)
    
    for i in range(len(arcArray)-len(node.getArcs())):
        lineArc = []
        for i in range(len(nodeArray)):
            lineArc.append(0)
        lineArc = np.array(lineArc)
        e.append(lineArc)
    e = np.array(e)
    arcs.append(e)
    
arcs = np.array(arcs)
dks = []
for node in nodeArray:
    dks.append(node.getCapacity())

In [183]:
from minilp import *
from minilp.solvers import scipy as scipy_solver

lp = problem('wildfire')

# create two integer variables
sigmas = lp.integer_var_list(len(nodeArray), 0, 24)
# landas = lp.integer_var_list(3,0,100)
pks = lp.integer_var_list(len(nodeArray),0,100)
Cmax = lp.integer_var(0,100)

#init evacuation tree
# e1 = np.array([[Arc(1,24,5),0,0],[Arc(1,24,5),0,0],[Arc(1,24,5),Arc(1,24,5),Arc(1,24,5)]])
# e2 = np.array([[0,Arc(1,24,5),0],[0,Arc(1,24,5),0],[Arc(1,24,5),Arc(1,24,5),Arc(1,24,5)]])
# e3 = np.array([[0,0,Arc(1,24,5)],[0,0,Arc(1,24,5)],[Arc(1,24,5),Arc(1,24,5),Arc(1,24,5)]])

# arcs = [e1,e2,e3]
# dks = [40,40,40]

# add constraints

#(4)
# for pk,landa,dk in zip(pks,landas,dks):
#     lp.add_constraint(pk*landa == dk)

#(5)
sik = np.zeros((len(arcs),arcs[0].shape[0]))
for k in range(len(arcs)):
    for i in range(arcs[k].shape[0]):
        if i == 0:
            if arcs[k][i][0] != 0:
                sik[k][i] = arcs[k][i][0].getT()
        else:
            if arcs[k][i][0] != 0:
                sik[k][i] = sik[k][i-1] + arcs[k][i][0].getT()
            
#(3)
for k in range(sik.shape[0]):
    for i in range(sik.shape[1]):
        if arcs[k][i][0] != 0:
            lp.add_constraint(sigmas[k] + sik[k][i]+ dks[k]*pks[k] <= arcs[k][i][0].getB())
        
#(2)
for k in range(sik.shape[0]):
    somSK = 0
    for i in range(sik.shape[1]):
        somSK = somSK + sik[k][i]
        
    lp.add_constraint(sigmas[k] + somSK + dks[k]*pks[k] <= Cmax)

    
# set the objective function
lp.set_objective(Cmax, sense=max)

# solve the problem
res = lp.lp_solve()
print(res)
print("sigma")
for i in range(len(sigmas)):
    print("sig",i,res.get_value(sigmas[i]))
print("Pk")
for i in range(len(pks)):
    print("Pk",i,res.get_value(pks[i]))

status = optimal, obj. = 100.0
sigma
sig 0 0.0
sig 1 0.0
sig 2 0.0
sig 3 0.0
sig 4 0.0
sig 5 0.0
sig 6 0.0
sig 7 0.0
sig 8 0.0
sig 9 0.0
Pk
Pk 0 0.009397590361445782
Pk 1 0.04947526236881558
Pk 2 0.010826771653543307
Pk 3 0.061224489795918366
Pk 4 0.007374631268436579
Pk 5 0.23648648648648646
Pk 6 0.0671785028790787
Pk 7 0.00893582453290008
Pk 8 0.016062413951353834
Pk 9 0.011908812521265737


In [175]:
class Arc:
    
    def __init__(self,t,b,c):
        self.t = t
        self.b = b
        self.c = c
        
    def getT(self):
        return self.t
    
    def getB(self):
        return self.b
    
    def getC(self):
        return self.c
        
    def isSame(self,arc):
        if self.t == arc.getT() and self.b == arc.getB() and self.c == arc.getC():
            return True
        else:
            return False

def mult(arcs1,arcs2):
    
    mat = np.zeros(arcs1.shape)
    
    for i in range(arcs1.shape[0]):
        for j in range(arcs1.shape[1]):
            if arcs1[i][j] != 0 and arcs2[i][j] != 0:
                if arcs1[i][j].isSame(arcs2[i][j]):
                    mat[i][j] = 1
                else:
                    mat[i][j] = 0
            else:
                mat[i][j] = 0
                
    return mat


In [177]:
from minilp import *
from minilp.solvers import scipy as scipy_solver
import numpy as np

lp = problem('wildfire')

# create two integer variables
sigmas = lp.integer_var_list(3, 0, 24)
# landas = lp.integer_var_list(3,0,100)
pks = lp.integer_var_list(3,0,100)
Cmax = lp.integer_var(0,100)

#init evacuation tree
e1 = np.array([[Arc(1,24,5),0,0],[Arc(1,24,5),0,0],[Arc(1,24,5),Arc(1,24,5),Arc(1,24,5)]])
e2 = np.array([[0,Arc(1,24,5),0],[0,Arc(1,24,5),0],[Arc(1,24,5),Arc(1,24,5),Arc(1,24,5)]])
e3 = np.array([[0,0,Arc(1,24,5)],[0,0,Arc(1,24,5)],[Arc(1,24,5),Arc(1,24,5),Arc(1,24,5)]])

arcs = [e1,e2,e3]
dks = [40,40,40]


#(4)
# for pk,landa,dk in zip(pks,landas,dks):
#     lp.add_constraint(pk*landa == dk)

#(5)
sik = np.zeros((len(arcs),arcs[0].shape[0]))
for k in range(len(arcs)):
    for i in range(arcs[k].shape[0]):
        if i == 0:
            sik[k][i] = arcs[k][i][k].getT()
        else:
            sik[k][i] = sik[k][i-1] + arcs[k][i][k].getT()
            
#(3)
for k in range(sik.shape[0]):
    for i in range(sik.shape[1]):
        lp.add_constraint(sigmas[k] + sik[k][i]+ dks[k]*pks[k] <= arcs[k][i][k].getB())
        
#(2)
for k in range(sik.shape[0]):
    somSK = 0
    for i in range(sik.shape[1]):
        somSK = somSK + sik[k][i]
        
    lp.add_constraint(sigmas[k] + somSK + dks[k]*pks[k] <= Cmax)

    
# set the objective function
lp.set_objective(Cmax, sense=max)

# solve the problem
res = lp.lp_solve()
print(res)
print("sigma")
for i in range(len(sigmas)):
    print("sig",i,res.get_value(sigmas[i]))
print("Pk")
for i in range(len(pks)):
    print("Pk",i,res.get_value(pks[i]))

status = optimal, obj. = 100.0
sigma
sig 0 0.0
sig 1 0.0
sig 2 0.0
Pk
Pk 0 0.525
Pk 1 0.525
Pk 2 0.525
